In [ ]:

import cv2
import numpy as np
from google.colab import files
import os

def detect_and_annotate_frame(frame):
    """
    Detects and classifies traffic lights in a single frame.

    Args:
        frame: The input video frame (as a NumPy array).

    Returns:
        The frame with detected traffic lights annotated with bounding boxes and labels.
    """
    # Convert frame from BGR to HSV color space for better color segmentation
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define HSV color ranges for red, yellow, and green

    lower_red1 = np.array([0, 120, 90])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 120, 90])
    upper_red2 = np.array([180, 255, 255])

    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([35, 255, 255])

    lower_green = np.array([40, 70, 70])
    upper_green = np.array([90, 255, 255])

    # Create binary masks for each color
    red_mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    red_mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    red_mask = cv2.add(red_mask1, red_mask2)
    yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    green_mask = cv2.inRange(hsv, lower_green, upper_green)

    # Store masks and their corresponding labels/colors
    masks = {"Red": red_mask, "Yellow": yellow_mask, "Green": green_mask}

    # Process each mask to find and draw contours
    for color_name, mask in masks.items():
        # Find contours (outlines of the colored regions)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            # --- Object Validation using size/shape constraints ---
            area = cv2.contourArea(contour)
            if area > 200: # Filter out very small, noisy detections
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = w / float(h)

                # Check for a somewhat circular/square shape
                if 0.5 < aspect_ratio < 1.8:
                    # Draw bounding box on the original frame
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    # Add a text label
                    cv2.putText(frame, color_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    return frame

# --- Main Video Processing Pipeline ---


input_video_path = 'traffic_test.mp4'
output_video_path = 'output_demo.mp4'

# Check if the input video file exists
if not os.path.exists(input_video_path):
    print(f"Error: Input file not found at '{input_video_path}'.")
    print("Please make sure you have uploaded the video and the filename is correct.")
else:
    # Open the video file for reading
    cap = cv2.VideoCapture(input_video_path)

    # Get video properties (width, height, frames per second)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Define the video codec and create a VideoWriter object to save the output

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    print(f"Processing video: '{input_video_path}'...")
    print("This may take a few moments depending on the video length.")

    # Loop through every frame of the video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            # End of video
            break

        # Process the frame to detect and annotate traffic lights
        processed_frame = detect_and_annotate_frame(frame)

        # Write the processed frame to the output video file
        out.write(processed_frame)

    # Release the video capture and writer objects
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    print("-" * 30)
    print("Video processing complete!")
    print(f"Annotated video saved as: '{output_video_path}'")
    print("-" * 30)

Processing video: 'traffic_test.mp4'...
This may take a few moments depending on the video length.
------------------------------
Video processing complete!
Annotated video saved as: 'output_demo.mp4'
------------------------------
